# Práctica 2.A Toma de contacto con AIMA

_GRUPO 7_

_Beatriz Herguedas Pinedo_

_Pablo Hernández Aguado_

La práctica está organizada en 3 partes. En la primera se muestra a través de ejemplos cómo se implementan algunos problemas clásicos como el de las jarras o el problema del ocho puzzle. En la segunda parte se muestra el uso de los algoritmos de búsqueda. En la tercera parte aprenderemos a medir las propiedades de los algoritmos.
En el notebook encontraras claramente identificados los lugares en los que debes incluir código o comentarios. 

## Parte I: Representación de problemas de espacios de estados.

### El primer paso es importar el código que necesitamos de search.py de AIMA y usar la clase Problem. En esta parte en vez de importarla la hemos copiado aquí para la explicación.

Como hemos visto en clase la representación de un problema de espacio de estados consiste en:
* Representar estados y acciones mediante una estructura de datos.
* Definir: estado_inicial, es_estado_final(_), acciones(_), aplica(_,_) y
  coste_de_aplicar_accion, si el problema tiene coste.

 La siguiente clase Problem representa este esquema general de cualquier
 problema de espacio de estados. Un problema concreto será una subclase de
 Problema, y requerirá implementar acciones, aplica y eventualmente __init__, actions,
 goal_test. La función coste_de_aplicar_accion la hemos incluido nosotros.

In [254]:
from search import *

In [255]:
   class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

Ahora vamos a ver un ejemplo de cómo definir un problema como subclase
de problema. En concreto, el problema de las jarras, visto en clase que es muy sencillo. 

In [256]:
class Jarras(Problem):
    """Problema de las jarras:
    Representaremos los estados como tuplas (x,y) de dos números enteros,
    donde x es el número de litros de la jarra de 4 e y es el número de litros
    de la jarra de 3"""

    def __init__(self):
        self.initial = (0,0)

    def actions(self,estado):
        jarra_de_4=estado[0]
        jarra_de_3=estado[1]
        accs=list()
        if jarra_de_4 > 0:
            accs.append("vaciar jarra de 4")
            if jarra_de_3 < 3:
                accs.append("trasvasar de jarra de 4 a jarra de 3")
        if jarra_de_4 < 4:
            accs.append("llenar jarra de 4")
            if jarra_de_3 > 0:
                accs.append("trasvasar de jarra de 3 a jarra de 4")
        if jarra_de_3 > 0:
            accs.append("vaciar jarra de 3")
        if jarra_de_3 < 3:
            accs.append("llenar jarra de 3")
        return accs

    def result(self,estado,accion):
        j4=estado[0]
        j3=estado[1]
        if accion=="llenar jarra de 4":
            return (4,j3)
        elif accion=="llenar jarra de 3":
            return (j4,3)
        elif accion=="vaciar jarra de 4":
            return (0,j3)
        elif accion=="vaciar jarra de 3":
            return (j4,0)
        elif accion=="trasvasar de jarra de 4 a jarra de 3":
            return (j4-3+j3,3) if j3+j4 >= 3 else (0,j3+j4)
        else: #  "trasvasar de jarra de 3 a jarra de 4"
            return (j3+j4,0) if j3+j4 <= 4 else (4,j3-4+j4)

    def goal_test(self,estado):
        return estado[0]==2


Vamos a probar algunos ejemplos.

In [257]:
p = Jarras()
p.initial

(0, 0)

In [258]:
p.actions(p.initial)

['llenar jarra de 4', 'llenar jarra de 3']

In [259]:
p.result(p.initial,"llenar jarra de 4")

(4, 0)

In [260]:
p.coste_de_aplicar_accion(p.initial,"llenar jarra de 4")

1

In [261]:
p.goal_test(p.initial)

False

# Representa el problema del puzzle de 8

Ahora vamos a definir la clase Ocho_Puzzle, que implementa la representación del problema del 8-puzzle visto en clase. 
Se os proporciona una versión incompleta y tendréis que completar el código que se presenta a continuación, en los lugares marcados con interrogantes.

### 8 Puzzle 

Tablero 3x3 cuyo objetivo es mover la configuración de las piezas desde un estado inicial dado a un estado objetivo moviendo las fichas al espacio en blanco. 

ejemplo:- 

                 Inicial                              Goal 
              | 7 | 2 | 4 |                       | 1 | 2 | 3 |
              | 5 | 0 | 6 |                       | 4 | 5 | 6 |
              | 8 | 3 | 1 |                       | 7 | 8 | 0 |
              
Hay 9! configuraciones iniciales pero ojo! porque no todas tienen solución. Tenlo en cuenta al hacer las pruebas. 

### EJERCICIO 1. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 

In [262]:
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self, estado):
        pos_hueco = estado.index(0) # busco la posicion del 0
        accs = list()
        
        if pos_hueco > 2: # not in (0,1,2)
            accs.append("Mover hueco arriba")
        
        if pos_hueco < 6: # not in (6,7,8)
            accs.append("Mover hueco abajo")
            
        if pos_hueco % 3 != 0: # not in (0,3,6)
            accs.append("Mover hueco izquierda")
        
        if pos_hueco % 3 != 2: # not in (2,5,8)
            accs.append("Mover hueco derecha")
        
        return accs
    
    ### EJERCICIO 1.1. COMPLETA LA DEFINICIÓN DE LOS OPERADORES.
    # OPERADOR 'actions' HECHO.

    def result(self, estado, accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco - 3]
            l[pos_hueco - 3] = 0
        
        if accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco + 3]
            l[pos_hueco + 3] = 0
        
        if accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco - 1]
            l[pos_hueco - 1] = 0
            
        if accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco + 1]
            l[pos_hueco + 1] = 0
        
        return tuple(l)
    
    ### EJERCICIO 1.2. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
    # OPERADOR 'result' HECHO.
    
    def h(self, node):
        """ Return the heuristic value for a given state. """
        return 1
    
    ## Redefinimos la función 'coste_de_aplicar_accion' para que dé error para
    ## acciones que no se pueden aplicar.
    
    def coste_de_aplicar_accion(self, estado, accion):
        if accion in self.actions(estado):
            return 1 ## El coste de mover el hueco es 1.
        else:
            return "No se puede aplicar esa acción."

### Ejercicio 1.3. Probar los siguientes ejemplos que se pueden ejecutar una vez se ha definido la clase:

In [263]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [264]:
# Movimientos disponibles a partir del estado inicial.
p8.actions(p8.initial)

## Respuesta: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

In [265]:
# Resultado de mover el hueco arriba.
p8.result(p8.initial,"Mover hueco arriba")

## Se intercambia el 0 con el 6.

(2, 8, 3, 1, 0, 4, 7, 6, 5)

In [266]:
# Resultado de mover el hueco abajo.
p8.result(p8.initial,"Mover hueco abajo")

## No se puede mover, pues el hueco está en la fila de abajo.
## Por ello mismo, "Mover hueco abajo" no aparecen en las acciones disponibles para el estado inicial.

IndexError: list index out of range

In [267]:
# Resultado de mover el hueco a la derecha.
p8.result(p8.initial,"Mover hueco derecha")

## Se intercambia el 0 con el 5.

(2, 8, 3, 1, 6, 4, 7, 5, 0)

In [268]:
p8.coste_de_aplicar_accion(p8.initial, "Mover hueco abajo")

'No se puede aplicar esa acción.'

## Parte II: Experimentación con los algoritmos implementados. Ejecución de los algoritmos de búsqueda de soluciones para una instancia del Problema.

### Usaremos búsqueda en anchura y en profundidad para encontrar soluciones tanto al problema de las jarras como al problema del ocho puzzle con distintos estados iniciales.

In [269]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

In [270]:
breadth_first_tree_search(Jarras()).solution()

['llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3',
 'vaciar jarra de 3',
 'trasvasar de jarra de 4 a jarra de 3',
 'llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3']

In [271]:
depth_first_graph_search(Jarras()).solution()

['llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'vaciar jarra de 4',
 'trasvasar de jarra de 3 a jarra de 4']

### Ejercicio 2. Prueba los algoritmos de búsqueda ciega con el puzzle de 8

#### Problema no resoluble.

In [272]:
p8 = EightPuzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [273]:
p8.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [21]:
breadth_first_tree_search(EightPuzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))).solution()
# Busqueda en anchura sin control de repetidos.  
# Busqueda en anchura es completo.. debería terminar...  ¿qué crees que está pasando?

## Este puzle no tiene solución, pues tiene un número impar de inversiones en su
## estado inicial:
## 2 -> 1                => 1 inv
## 8 -> 3, 1, 6, 4, 7, 5 => 6 inv
## 3 -> 1                => 1 inv
## 1 ->                  => 0 inv
## 6 -> 4, 5             => 2 inv
## 4 ->                  => 0 inv
## 7 -> 5                => 1 inv
## ______________________ : _____
## Número de inversiones => 11 (impar)
##
## Por ello, aunque la búsqueda sea completa, esta no termina.
## Si la búsqueda tuviese control de repetidos sí que terminaría, pues
## recorrería todas las posibles combinaciones del puzle sin que ninguna
## fuese el estado objetivo.

In [274]:
# Comprobamos la existencia de solución con 'check_solvability'.
p8.check_solvability(p8.initial)

False

In [275]:
##breadth_first_graph_search(EightPuzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))).solution()
## En efecto, con control de repetidos termina, pero tarda mucho al tener que
## recorrer todas las posibles combinaciones del puzle.

#### Problema resoluble.

In [276]:
## Con el estado inicial (1,2,3,4,5,6,0,7,8), el algoritmo devuelve la solución
## correcta, pues el puzle es resoluble.
breadth_first_tree_search(Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))).solution()

['Mover hueco derecha', 'Mover hueco derecha']

In [277]:
%%timeit
breadth_first_tree_search(Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))).solution()

131 µs ± 1.37 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Aquí hemos comprobado el tiempo medio que tarda la búsqueda en anchura: 131 µs.

In [278]:
depth_first_tree_search(Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))).solution()

['Mover hueco derecha', 'Mover hueco derecha']

In [393]:
%%timeit
depth_first_tree_search(Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))).solution()

37 µs ± 349 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Seguidamente, el timepo que tarda la búsqueda en profundidad por cada iteración: 37 µs; tiempo menor que el de la búsqueda en anchura.

#### ---------------------------------------------------------------------------------------------------------------

A continuación, definimos un puzle con estado inicial (2,4,3,1,5,6,7,8,0), resoluble, y comprobamos el funcionamiento de todos los algoritmos de búsqueda ciega, así como su tiempo de ejecución.

In [279]:
estado = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))

#### Búsqueda en anchura (sin control de repetidos).

In [18]:
breadth_first_tree_search(estado).solution()
# Respuesta: ['UP', 'LEFT', 'UP', 'LEFT', 'DOWN', 'RIGHT', 'RIGHT', 'DOWN']

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [19]:
%%timeit
sol_bt = breadth_first_tree_search(estado)

41.3 ms ± 733 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [402]:
## Con búsqueda en anchura (sin control de repetidos) tarda de media 43 ms.

#### Búsqueda an anchura (con control de repetidos).

In [20]:
breadth_first_graph_search(estado).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [21]:
%%timeit
breadth_first_graph_search(estado)

4.54 ms ± 222 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [406]:
## Con búsqueda en anchura (con control de repetidos) tarda de media 4.67 ms.

#### Búsqueda en profundidad (sin control de repetidos).

In [ ]:
#### !!! Tarda demasiado.
## %%time
## depth_first_tree_search(estado).solution()

#### Búsqueda en profundidad (con control de repetidos).

In [253]:
%%time
depth_first_graph_search(estado).solution()

Wall time: 21min 37s


['Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover h

In [280]:
## Con búsqueda en profundidad (con control de repetidos) tarda en una ejecución 21 minutos y 37 segundos.
## Por tanto, la búsqueda en profundidad sin control de repetidos será mucho más costosa, y no merece la pena dejar que acabe.

#### En este ejercicio se ha podido observar los resultados y tiempo de la ejecución de los algoritmos de búsqueda ciega.  Escribe aquí tus conclusiones:

La búsqueda ciega, al ser completa, funciona para encontrar las soluciones a los 8-puzles resolubles. Así, consigue una solución en poco tiempo, tanto en anchura como en profundidad, para el estado inicial (1, 2, 3, 4, 5, 6, 0, 7, 8).

No obstante, para el puzle con estado inicial (2, 4, 3, 1, 5, 6, 7, 8, 0), aunque la búsqueda en anchura consigue la solución rápidamente en cuestión de milisegundos, no lo hace así la búsqueda en profundidad, que tarda hasta 20 minutos con control de repetidos.

### Ejercicio 3: definir al menos las siguientes funciones heurísticas para el 8 puzzle:
* linear(node): cuenta el número de casillas mal colocadas respecto al estado final.
* manhattan(node): suma la distancia Manhattan desde cada casilla a la posición en la que debería estar en el estado final.
* max_heuristic(node: maximo de las dos anteriores
* sqrt_manhattan(node):  raíz cuadrada de la distancia Manhattan

In [281]:
# Heuristicas para el 8 Puzzle 

def linear(node):
    #goal = node.state.goal
    goal = (1,2,3,4,5,6,7,8,0)
    
    cont = 0
    for i in range(0,9): # Posición i.
        if node.state[i] != 0: # No hueco.
            if node.state[i] != goal[i]: # Piezas distintas en misma posición.
                cont += 1
    
    return cont

def manhattan(node):
    state = node.state
    goal = (1,2,3,4,5,6,7,8,0)
    
    cont = 0
    for i in range(0,9):
        if node.state[i] != 0:
            j = goal.index(state[i])
            if i != j:
                filEstado = i // 3
                colEstado = i % 3
                filFinal  = j // 3
                colFinal  = j % 3
                
                cont += abs(filEstado - filFinal) + abs(colEstado - colFinal)
    
    return cont

def sqrt_manhattan(node):
    ##state = node.state
    
    return math.sqrt(manhattan(node))

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)

### Ejercicio 4. 
Usar las implementaciones de los algoritmos que correspondan a búsqueda_coste_uniforme, busqueda_primero_el_mejor y búsqueda_a_estrella (con las heurísticas anteriores) para resolver el problema del 8 puzzle para el siguiente estado inicial y comparar los costes temporales usando %timeit y comentar los resultados.

              +---+---+---+
              | 2 | 4 | 3 |
              +---+---+---+
              | 1 | 5 | 6 |
              +---+---+---+
              | 7 | 8 | H |
              +---+---+---+


In [282]:
puzle = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))

In [283]:
puzle.initial

(2, 4, 3, 1, 5, 6, 7, 8, 0)

In [284]:
puzle.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

#### Búsqueda de coste uniforme.

In [27]:
uniform_cost_search(puzle).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [29]:
%%timeit
uniform_cost_search(puzle)

21.9 ms ± 429 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
## Con búsqueda en coste uniforme tarda de media 21.9 ms.

#### Búsqueda 'Primero el mejor' con heurística 'linear'.

In [30]:
best_first_graph_search(puzle, linear).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [32]:
%%timeit
best_first_graph_search(puzle, linear)

148 ms ± 973 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
## Con búsqueda 'Primero el mejor' con heurística 'linear' tarda de media 148 ms.

#### Búsqueda 'Primero el mejor' con heurística 'manhattan'.

In [34]:
best_first_graph_search(puzle, manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [87]:
%%timeit
best_first_graph_search(puzle, manhattan)

566 µs ± 5.06 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [88]:
## Con búsqueda 'Primero el mejor' con heurística 'manhattan' tarda de media 566 µs ~ 0.57 ms.

#### Búsqueda 'Primero el mejor' con heurística 'sqrt_manhattan'.

In [37]:
best_first_graph_search(puzle, sqrt_manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [45]:
%%timeit
best_first_graph_search(puzle, sqrt_manhattan)

594 µs ± 5.34 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [46]:
## Con búsqueda 'Primero el mejor' con heurística 'sqrt_manhattan' tarda de media 594 µs ~ 0.59 ms.

#### Búsqueda 'Primero el mejor' con heurística 'max_heuristic'.

In [84]:
best_first_graph_search(puzle, max_heuristic).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [90]:
%%timeit
best_first_graph_search(puzle, max_heuristic)

664 µs ± 5.97 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
## Con búsqueda 'Primero el mejor' con heurística 'max_heuristic' tarda de media 664 µs ~ 0.66 ms.

#### Búsqueda A* (sin heurística)

In [51]:
astar_search(puzle).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [56]:
%%timeit
astar_search(puzle)

23.2 ms ± 488 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
## Con búsqueda A* sin heurística tarda de media 23.2 ms.

#### Búsqueda A* con heurística 'linear'.

In [57]:
astar_search(puzle, linear).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [61]:
%%timeit
astar_search(puzle, linear)

638 µs ± 6.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [62]:
## Con búsqueda A* con heurística 'linear' tarda de media 638 µs ~ 0.64 ms.

#### Búsqueda A* con heurística 'manhattan'.

In [63]:
astar_search(puzle, manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [65]:
%%timeit
astar_search(puzle, manhattan)

608 µs ± 5.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [66]:
## Con búsqueda A* con heurística 'manhattan' tarda de media 608 µs ~ 0.61 ms.

#### Búsqueda A* con heurística 'sqrt_manhattan'.

In [67]:
astar_search(puzle, sqrt_manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [71]:
%%timeit
astar_search(puzle, sqrt_manhattan)

3.34 ms ± 52.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [72]:
## Con búsqueda A* con heurística 'sqrt_manhattan' tarda de media 3.34 ms.

#### Búsqueda A* con heurística 'max_heuristic'.

In [73]:
astar_search(puzle, max_heuristic).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [75]:
%%timeit
astar_search(puzle, max_heuristic)

699 µs ± 5.09 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [76]:
## Con búsqueda A* con heurística 'max_heuristic' tarda de media 699 µs ~ 0.70 ms.

#### Tabla de resultados para el estado inicial (2, 4, 3, 1, 5, 6, 7, 8, 0).

|                         Método | Tiempo (µs) |
|:-------------------------------|:------------|
|                 Coste Uniforme |       21900 |
|         Primero Mejor (linear) |      148000 |
|      Primero Mejor (manhattan) |         566 |
| Primero Mejor (sqrt_manhattan) |         594 |
|  Primero Mejor (max_heuristic) |         664 |
|                             A* |       23200 |
|                    A* (linear) |         638 |
|                 A* (manhattan) |         608 |
|            A* (sqrt_manhattan) |        3340 |
|             A* (max_heuristic) |         699 |

#### ---------------------------------------------------------------------------------------------------------------

¿Has notado diferencias en los tiempos de ejecución? Vamos a medirlo. Aunque las heurísticas no afectan a la solución obtenida sí hay diferencias importantes en el tiempo de cálculo
<br>

In [285]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
puzzle_2 = Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))
puzzle_3 = Ocho_Puzzle((1, 2, 3, 4, 5, 7, 8, 6, 0))

In [103]:
%%timeit -n 200 -r 20
astar_search(puzzle_1, linear)
astar_search(puzzle_2, linear)
astar_search(puzzle_3, linear)

7.1 ms ± 102 µs per loop (mean ± std. dev. of 20 runs, 200 loops each)


In [ ]:
## Con búsqueda A* con heurística 'linear' tarda de media 7.1 ms.

In [104]:
%%timeit -n 200 -r 20
astar_search(puzzle_1, manhattan)
astar_search(puzzle_2, manhattan)
astar_search(puzzle_3, manhattan)

4.28 ms ± 178 µs per loop (mean ± std. dev. of 20 runs, 200 loops each)


In [105]:
## Con búsqueda A* con heurística 'manhattan' tarda de media 4.28 ms.

In [106]:
%%timeit -n 200 -r 20
astar_search(puzzle_1, sqrt_manhattan)
astar_search(puzzle_2, sqrt_manhattan)
astar_search(puzzle_3, sqrt_manhattan)

63.3 ms ± 844 µs per loop (mean ± std. dev. of 20 runs, 200 loops each)


In [ ]:
## Con búsqueda A* con heurística 'sqrt_manhattan' tarda de media 63.3 ms.

In [109]:
%%timeit -n 200 -r 20
astar_search(puzzle_1, max_heuristic)
astar_search(puzzle_2, max_heuristic)
astar_search(puzzle_3, max_heuristic)

4.55 ms ± 118 µs per loop (mean ± std. dev. of 20 runs, 200 loops each)


In [108]:
## Con búsqueda A* con heurística 'max_heuristic' tarda de media 4.81 ms.

#### Tabla de resultados para las heurísticas.

|                         Método | Tiempo (µs) |
|:-------------------------------|:------------|
|                    A* (linear) |        7100 |
|                 A* (manhattan) |        4280 |
|            A* (sqrt_manhattan) |       63300 |
|             A* (max_heuristic) |        4810 |

#### Escribe aquí tus conclusiones sobre qué heurística es mejor y por qué.

La mejor heurística es la Manhattan, que supera por poco a la Lineal (en nuestros resultados; en los resultados del enunciado dan números parecidos). 

Respecto a la heurística Máximo, que coge el máximo entre la heurística Lineal y la Manhattan, esta da un resultado parecido a la Manhattan. Precisamente, puesto que la Lineal cuenta el número de piezas descoladas, si este es N, entonces la suma de las distancias Manhattan de las piezas a sus posiciones será por lo menos N; por lo que 'max_heuristic' siempre elige el valor de la heurística Manhattan.

Por otro lado, la heurística que toma la raíz cuadrada de la Manhattan es claramente menos eficiente que el resto.

## Parte III:  Calcular estadísticas sobre la ejecución de los algoritmos para resolución de problemas de ocho puzzle.

### El objetivo es comprobar experimentalmente las propiedades teóricas de los algoritmos vistas en clase.

Usaremos la función %timeit para medir los tiempos y, para el espacio, una version modificada de Problema que almacena el número de nodos.

En la clase modificada también vamos a incluir un cálculo que nos diga si cierto tablero tiene solución o no. Esto es muy útil... como hemos comentado al principio solo algunos tableros tienen solucion.
En el caso del puzle de 8 se puede comprobar si un tablero es resoluble calculando el número de inversiones. Si es impar el tablero no tiene solución. 

> The solvability of a configuration can be checked by calculating the Inversion Permutation. If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.

In [286]:
# Hacemos una definición ampliada de la clase Problem de AIMA que nos va a permitir experimentar con distintos
# estados iniciales, algoritmos y heurísticas, para resolver el 8-puzzle. 
# Añadimos en la clase ampliada la capacidad para contar el número de nodos analizados durante la
# búsqueda:


class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado, accion)
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """

        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        
        return inversion % 2 == 0        

#### Solución de un problema con analizados.

In [287]:
estado_inicial = (1,2,3,4,5,6,7,0,8)
p8 = Ocho_Puzzle(estado_inicial)
p8p = Problema_con_Analizados(p8)

In [288]:
print(p8p.initial)
print(p8p.goal)

(1, 2, 3, 4, 5, 6, 7, 0, 8)
(1, 2, 3, 4, 5, 6, 7, 8, 0)


In [289]:
p8p.analizados

0

In [ ]:
##Inicialmente, el número de nodos analizados es 0, pues el problema no se ha tratado.

In [290]:
astar_search(p8p, linear).solution()

['Mover hueco derecha']

In [291]:
p8p.analizados

2

In [ ]:
## Tras resolver el problema con búsqueda A* (linear), vemos que este método ha analizado 2 nodos.

#### Función de resolución con información.

In [293]:
def resuelve_ocho_puzzle(estado_inicial, algoritmo, h=None, p=True):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:
           puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
           resuelve_ocho_puzzle(puzzle_1,astar_search,h2_ocho_puzzle)
        Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
        'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
        Algoritmo: astar_search
        Heurística: h2_ocho_puzzle
        Longitud de la solución: 8. Nodos analizados: 11
       """

    p8p = Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
    if p8p.check_solvability(estado_inicial):
        if h: 
            sol = algoritmo(p8p,h).solution()
        else: 
            sol = algoritmo(p8p).solution()
            
        if p:
            print("Solución: {0}".format(sol))
            print("Algoritmo: {0}".format(algoritmo.__name__))
            
            if h: 
                print("Heurística: {0}".format(h.__name__))
            else:
                pass
            
            print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
        
    else: 
        print("Este problema no tiene solucion. ")


### Ejercicio 5.

Intentar resolver usando las distintas búsquedas y, en su caso, las distintas heurísticas, el problema del 8-puzzle para los siguientes estados iniciales.

E1 = (2,1,3,4,8,6,7,0,5)

E2 = (1,0,3,4,8,6,7,2,5)

E3 = (4,5,6,1,0,3,7,8,2)

E4 = (1,2,3,0,5,6,4,7,8)


Se pide, en cada caso, obtener detalles del tiempo y espacio necesario para la resolución de estos estados. Hacerlo con la función 'resuelve_ocho_puzzle', para obtener, además de la solución, la longitud (el coste) de la solución obtenida y el número de nodos analizados. Anotar los resultados en la siguiente tabla y justificarlos con las distintas propiedades teóricas estudiadas.

|            Estado inicial | E1       | E2       | E3       | E4       |
|:--------------------------|:---------|:---------|:---------|:---------|
|                   Anchura | L = ___  | L = ___  | L = ___  | L = ___  |
|                           | T = ___  | T = ___  | T = ___  | T = ___  |
|                           | NA = ___ | NA = ___ | NA = ___ | NA = ___ |
|               Profundidad | L = ___  | L = ___  | L = ___  | L = ___  |
|                           | T = ___  | T = ___  | T = ___  | T = ___  |
|                           | NA = ___ | NA = ___ | NA = ___ | NA = ___ |
|            Coste Uniforme | L = ___  | L = ___  | L = ___  | L = ___  |
|                           | T = ___  | T = ___  | T = ___  | T = ___  |
|                           | NA = ___ | NA = ___ | NA = ___ | NA = ___ |
|    Primero mejor (linear) | L = ___  | L = ___  | L = ___  | L = ___  |
|                           | T = ___  | T = ___  | T = ___  | T = ___  |
|                           | NA = ___ | NA = ___ | NA = ___ | NA = ___ |
| Primero mejor (manhattan) | L = ___  | L = ___  | L = ___  | L = ___  |
|                           | T = ___  | T = ___  | T = ___  | T = ___  |
|                           | NA = ___ | NA = ___ | NA = ___ | NA = ___ |
|               A* (linear) | L = ___  | L = ___  | L = ___  | L = ___  |
|                           | T = ___  | T = ___  | T = ___  | T = ___  |
|                           | NA = ___ | NA = ___ | NA = ___ | NA = ___ |
|            A* (manhattan) | L = ___  | L = ___  | L = ___  | L = ___  |
|                           | T = ___  | T = ___  | T = ___  | T = ___  |
|                           | NA = ___ | NA = ___ | NA = ___ | NA = ___ |

L = longitud de la solución

NA =  nodos analizados

T = tiempo (en µs)



#### Estados iniciales.

In [294]:
E1 = (2,1,3,4,8,6,7,0,5)
E2 = (1,0,3,4,8,6,7,2,5)
E3 = (4,5,6,1,0,3,7,8,2)
E4 = (1,2,3,0,5,6,4,7,8)

* Se ha añadido un argumento 'p' a la función 'resuelve_ocho_puzzle' para que no muestre por pantalla los resultados. De esta forma, al utilizar %%timeit para medir el tiempo, no muestra los resultados cada vez que %%timeit ejecuta la función.

#### Búsqueda en anchura.

In [119]:
resuelve_ocho_puzzle(E1, breadth_first_graph_search)

Solución: ['Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 17. Nodos analizados: 13634


In [246]:
%%timeit
resuelve_ocho_puzzle(E1, breadth_first_graph_search, None, False)

15.1 s ± 205 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [121]:
## Búsqueda en anchura E1
## 17 acciones
## 13634 nodos analizados
## 15.1 s de media
## ------------------------------------------

In [122]:
resuelve_ocho_puzzle(E2, breadth_first_graph_search)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 11. Nodos analizados: 1255


In [247]:
%%timeit
resuelve_ocho_puzzle(E2, breadth_first_graph_search, None, False)

137 ms ± 1.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
## Búsqueda en anchura E2
## 11 acciones
## 1255 nodos analizados
## 137 ms de media
## ------------------------------------------

In [248]:
%%time
resuelve_ocho_puzzle(E3, breadth_first_graph_search)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 20. Nodos analizados: 57057
Wall time: 4min 26s


In [128]:
## Búsqueda en anchura E3
## 20 acciones
## 57057 nodos analizados
## 4 min 26 s en 1 ejecución
## ------------------------------------------

In [129]:
resuelve_ocho_puzzle(E4, breadth_first_graph_search)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 3. Nodos analizados: 13


In [130]:
%%timeit
resuelve_ocho_puzzle(E4, breadth_first_graph_search, None, False)

162 µs ± 1.12 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
## Búsqueda en anchura E4.
## 3 acciones
## 13 nodos analizados
## 162 microsegundos de media
## ------------------------------------------

#### Búsqueda en profundidad.

In [249]:
%%time
resuelve_ocho_puzzle(E1, depth_first_graph_search)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco der

In [ ]:
## Búsqueda en profundidad E1
## 7977 acciones
## 8198 nodos analizados
## 25.5 s en 1 ejecución.
## --------------------------------------------

In [132]:
#### !!! Tarda demasiado.
## %%time
## resuelve_ocho_puzzle(E2, depth_first_graph_search)

In [ ]:
## Búsqueda en profundidad E2
##  acciones
##  nodos analizados
## 
## -------------------------------------------

In [250]:
%%time
resuelve_ocho_puzzle(E3, depth_first_graph_search)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquie

In [ ]:
## Búsqueda en profundidad E3
## 39720 acciones
## 42926 nodos analizados
## 10 min 54 s en 1 ejecución.
## -------------------------------------------

In [133]:
resuelve_ocho_puzzle(E4, depth_first_graph_search)

Solución: ['Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: depth_first_graph_search
Longitud de la solución: 27. Nodos analizados: 28


In [136]:
%%timeit
resuelve_ocho_puzzle(E4, depth_first_graph_search, None, False)

868 µs ± 29.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
## Búsqueda en profundidad E4
## 27 acciones
## 28 nodos analizados
## 868 microsegundos de media
## -------------------------------------------

#### Búsqueda de coste uniforme.

In [251]:
%%time
resuelve_ocho_puzzle(E1, uniform_cost_search)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: uniform_cost_search
Longitud de la solución: 17. Nodos analizados: 14092
Wall time: 2min 16s


In [ ]:
## Coste uniforme E1
## 17 acciones
## 14092 nodos analizados
## 2 min 16 s en 1 ejecución
## -------------------------------------------

In [301]:
resuelve_ocho_puzzle(E2, uniform_cost_search)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: uniform_cost_search
Longitud de la solución: 11. Nodos analizados: 870


In [303]:
%%timeit
resuelve_ocho_puzzle(E2, uniform_cost_search, None, False)

423 ms ± 21.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
## Coste uniforme E2
## 11 acciones
## 870 nodos analizados
## 423 milisegundos de media
## -------------------------------------------

In [252]:
%%time
resuelve_ocho_puzzle(E3, uniform_cost_search)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: uniform_cost_search
Longitud de la solución: 20. Nodos analizados: 48428
Wall time: 24min 31s


In [ ]:
## Coste uniforme E3
## 20 acciones
## 48428 nodos analizados
## 24 min 31 s en 1 ejecución
## -------------------------------------------

In [306]:
resuelve_ocho_puzzle(E4, uniform_cost_search)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: uniform_cost_search
Longitud de la solución: 3. Nodos analizados: 13


In [308]:
%%timeit
resuelve_ocho_puzzle(E4, uniform_cost_search, None, False)

592 µs ± 40.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
## Coste uniforme E4
## 3 acciones
## 13 nodos analizados
## 592 microsegundos de media
## -------------------------------------------

#### Búsqueda 'Primero mejor' con heurística 'linear'.

In [309]:
resuelve_ocho_puzzle(E1, best_first_graph_search, linear)

Solución: ['Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: best_first_graph_search
Heurística: linear
Longitud de la solución: 39. Nodos analizados: 171

In [310]:
%%timeit
resuelve_ocho_puzzle(E1, best_first_graph_search, linear, False)

23.5 ms ± 1.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
## Primero mejor 'linear' E1
## 39 acciones
## 171 nodos analizados
## 23.5 milisegundos de media
## -------------------------------------------

In [311]:
resuelve_ocho_puzzle(E2, best_first_graph_search, linear)

Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: best_first_graph_search
Heurística: linear
Longitud de la solución: 33. Nodos analizados: 78


In [312]:
%%timeit
resuelve_ocho_puzzle(E2, best_first_graph_search, linear, False)

7.01 ms ± 229 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
## Primero mejor 'linear' E2
## 33 acciones
## 78 nodos analizados
## 7.01 milisegundos de media
## -------------------------------------------

In [313]:
resuelve_ocho_puzzle(E3, best_first_graph_search, linear)

Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hue

In [315]:
%%timeit
resuelve_ocho_puzzle(E3,best_first_graph_search, linear, False)

315 ms ± 16.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
## Primero mejor 'linear' E3
## 76 acciones
## 747 nodos analizados
## 315 milisegundos de media
## -------------------------------------------

In [316]:
resuelve_ocho_puzzle(E4, best_first_graph_search, linear)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: best_first_graph_search
Heurística: linear
Longitud de la solución: 3. Nodos analizados: 4


In [318]:
%%timeit
resuelve_ocho_puzzle(E4, best_first_graph_search, manhattan, False)

243 µs ± 36 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
## Primero mejor 'linear' E4
## 3 acciones
## 4 nodos analizados
## 243 microsegundos de media
## -------------------------------------------

#### Búsqueda 'Primero mejor' con heurística 'manhattan'.

In [319]:
resuelve_ocho_puzzle(E1, best_first_graph_search, manhattan)

Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hu

In [320]:
%%timeit
resuelve_ocho_puzzle(E1, best_first_graph_search, manhattan, False)

174 ms ± 4.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
## Primero mejor 'manhattan' E1
## 49 acciones
## 522 nodos analizados
## 174 milisegundos de media
## -------------------------------------------

In [321]:
resuelve_ocho_puzzle(E2, best_first_graph_search, manhattan)

Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo']
Algoritmo: best_first_graph_search
Heurística: manhattan
Longitud de la solución: 29. Nodos analizados: 75


In [322]:
%%timeit
resuelve_ocho_puzzle(E2, best_first_graph_search, manhattan, False)

7.17 ms ± 346 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
## Primero mejor 'manhattan' E2
## 29 acciones
## 75 nodos analizados
## 7.17 milisegundos de media
## -------------------------------------------

In [323]:
resuelve_ocho_puzzle(E3, best_first_graph_search, manhattan)

Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: best_first_graph_search
Heurística: manhattan
Longitud de la solución: 36. Nodos analizados: 401


In [324]:
%%timeit
resuelve_ocho_puzzle(E3, best_first_graph_search, manhattan, False)

118 ms ± 5.91 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
## Primero mejor 'manhattan' E3
## 36 acciones
## 401 nodos analizados
## 118 milisegundos de media
## -------------------------------------------

In [325]:
resuelve_ocho_puzzle(E4, best_first_graph_search, manhattan)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: best_first_graph_search
Heurística: manhattan
Longitud de la solución: 3. Nodos analizados: 4


In [326]:
%%timeit
resuelve_ocho_puzzle(E4, best_first_graph_search, manhattan, False)

187 µs ± 8.21 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
## Primero mejor 'manhattan' E4
## 3 acciones
## 4 nodos analizados
## 187 microsegundos de media
## -------------------------------------------

#### Búsqueda A* con heurística 'linear'.

In [327]:
resuelve_ocho_puzzle(E1, astar_search, linear)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 17. Nodos analizados: 835


In [328]:
%%timeit
resuelve_ocho_puzzle(E1, astar_search, linear, False)

404 ms ± 30.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
## Búsqueda A* 'linear' E1
## 17 acciones
## 835 nodos analizados
## 404 milisegundos de media
## -------------------------------------------

In [329]:
resuelve_ocho_puzzle(E2, astar_search, linear)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 11. Nodos analizados: 74


In [330]:
%%timeit
resuelve_ocho_puzzle(E2, astar_search, linear, False)

7.67 ms ± 264 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
## Búsqueda A* 'linear' E2
## 11 acciones
## 74 nodos analizados
## 7.67 milisegundos de media
## -------------------------------------------

In [331]:
resuelve_ocho_puzzle(E3, astar_search, linear)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 20. Nodos analizados: 3230


In [332]:
%%timeit
resuelve_ocho_puzzle(E3, astar_search, linear, False)

5.34 s ± 210 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
## Búsqueda A* 'linear' E3
## 20 acciones
## 3230 nodos analizados
## 5.34 segundos de media
## -------------------------------------------

In [333]:
resuelve_ocho_puzzle(E4, astar_search, linear)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 3. Nodos analizados: 4


In [334]:
%%timeit
resuelve_ocho_puzzle(E4, astar_search, linear, False)

174 µs ± 2.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
## Búsqueda A* 'linear' E4
## 3 acciones
## 4 nodos analizados
## 174 microsegundos de media
## -------------------------------------------

#### Búsqueda A* con heurística 'manhattan'.

In [335]:
resuelve_ocho_puzzle(E1, astar_search, manhattan)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 17. Nodos analizados: 245


In [336]:
%%timeit
resuelve_ocho_puzzle(E1, astar_search, manhattan, False)

43 ms ± 2.96 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
## Búsqueda A* 'manhattan' E1
## 17 acciones
## 245 nodos analizados
## 43 milisegundos de media
## -------------------------------------------

In [337]:
resuelve_ocho_puzzle(E2, astar_search, manhattan)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 11. Nodos analizados: 24


In [339]:
%%timeit
resuelve_ocho_puzzle(E2, astar_search, manhattan, False)

1.62 ms ± 59 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
## Búsqueda A* 'manhattan' E2
## 11 acciones
## 24 nodos analizados
## 1.62 milisegundos de media
## -------------------------------------------

In [340]:
resuelve_ocho_puzzle(E3, astar_search, manhattan)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 20. Nodos analizados: 732


In [342]:
%%timeit
resuelve_ocho_puzzle(E3, astar_search, manhattan, False)

289 ms ± 13.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
## Búsqueda A* 'manhattan' E3
## 20 acciones
## 732 nodos analizados
## 289 milisegundos de media
## -------------------------------------------

In [344]:
resuelve_ocho_puzzle(E4, astar_search, manhattan)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 3. Nodos analizados: 4


In [343]:
%%timeit
resuelve_ocho_puzzle(E4, astar_search, manhattan, False)

219 µs ± 27.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
## Búsqueda A* 'manhattan' E4
## 3 acciones
## 4 nodos analizados
## 219 microsegundos de media
## -------------------------------------------

#### Tabla actualizada.

|              Estado inicial |        | E1     | E2     | E3      | E4     |
|----------------------------:|:-------|-------:|-------:|--------:|-------:|
|                   _Anchura_ | **L**  | 17     | 11     | 20      | 3      |
|                             | **T**  | 15100  | 137    | 266000  | 0.162  |
|                             | **NA** | 13634  | 1255   | 57057   | 13     |
|               _Profundidad_ | **L**  | 7977   | NAN    | 39720   | 27     |
|                             | **T**  | 25500  | NAN    | 654000  | 0.868  |
|                             | **NA** | 8198   | NAN    | 42926   | 28     |
|            _Coste uniforme_ | **L**  | 17     | 11     | 20      | 3      |
|                             | **T**  | 136000 | 423    | 1471000 | 0.592  |
|                             | **NA** | 14092  | 870    | 48428   | 13     |
|    _Primero mejor (linear)_ | **L**  | 39     | 33     | 76      | 3      |
|                             | **T**  | 23.500 | 7.010  | 315     | 0.243  |
|                             | **NA** | 171    | 78     | 747     | 4      |
| _Primero mejor (manhattan)_ | **L**  | 49     | 29     | 36      | 3      |
|                             | **T**  | 174    | 7.170  | 118     | 0.187  |
|                             | **NA** | 522    | 75     | 401     | 118    |
|               _A* (linear)_ | **L**  | 17     | 11     | 20      | 3      |
|                             | **T**  | 404    | 7.670  | 5340    | 0.174  |
|                             | **NA** | 835    | 74     | 3230    | 4      |
|            _A/ (manhattan)_ | **L**  | 17     | 11     | 20      | 3      |
|                             | **T**  | 43     | 1.620  | 289     | 0.219  |
|                             | **NA** | 245    | 24     | 732     | 4      |

### Ejercicio 6: [OPCIONAL] Definir una heurística más informada y completa la tabla anterior para ver cómo afecta al número de nodos generados por los algoritmos

Proponemmos como heurística h3 la se obtiene sumando a la heurística manhattan una componente que cuantifica la "secuencialidad" en las casillas de un tablero, al recorrerlo en el sentido de las aguas del reloj ¿Es h3 admisble? Comprobar cómo se comporta esta heurística cuando se usa en A*, con cada uno de los estados anteriores. Comentar los resultados. Puedes proponer otra heuristica siempre que sea más informada que todas las heurísticas ya utilizadas.


#### Heurística 'sequential'.

Nuestro estado objetivo viene dado por la configuración del puzzle:

              +---+---+---+
              | 1 | 2 | 3 |
              +---+---+---+
              | 4 | 5 | 6 |
              +---+---+---+
              | 7 | 8 | H |
              +---+---+---+

Ahora, definimos un orden secuencial en el tablero, en el sentido de las agujas del reloj:

              +---+---+---+
              | 4 | 5 | 6 |
              +---+---+---+
              | 3 | 0 | 7 | >> Representa un orden, no un estado del puzzle.
              +---+---+---+
              | 2 | 1 | 8 |
              +---+---+---+
              
Así, tomando las posiciones del vector que representa la tabla:

              +---+---+---+
              | 0 | 1 | 2 |
              +---+---+---+
              | 3 | 4 | 5 |
              +---+---+---+
              | 6 | 7 | 8 |
              +---+---+---+

| Posición | Adyacente |
|:--------:|:---------:|
|        0 |         1 |
|        1 |         2 |
|        2 |         5 |
|        3 |         0 |
|        4 |         7 |
|        5 |         8 |
|        6 |         3 |
|        7 |         6 |
|        8 |         - |

De esta forma, hacemos:

* Si una pieza ocupa el lugar donde debe estar el hueco (seq. 8), entonces puntúa 1.
* Para cualquier otra pieza que no ocupe el hueco (seq. 0-7), si su adyacente (seq.) no es la correcta, entonces se puntúa 2.

In [306]:
def nextPosition(pos):
    switcher = {
        0: 1,
        1: 2,
        2: 5,
        3: 0,
        4: 7,
        5: 8,
        6: 3,
        7: 6,
    }
    
    return switcher.get(pos, -1)

def nextPiece(piece):
    switcher = {
        1: 2,
        2: 3,
        3: 6,
        4: 1,
        5: 8,
        6: 0,
        7: 4,
        8: 7
    }
    
    return switcher.get(piece, -1)

def sequential(node, l=1):
    state = node.state
    goal = (1,2,3,4,5,6,7,8,0)
    
    score = 0
    for piece in range(1,9):
        
        pos = state.index(piece)
        
        if pos == 8:
            score += 1*l
        else:
            nextPos = nextPosition(pos) ## next sequential position
            adjPiece = state[nextPos]
            
            if nextPiece(piece) != adjPiece:
                score += 2*l
                
    return score

Por ejemplo, para el estado inicial dado por:

              +---+---+---+
              | 2 | 1 | 3 |
              +---+---+---+
              | 4 | 5 | 6 |
              +---+---+---+
              | 7 | 8 | 0 |
              +---+---+---+
              
su puntuación es 6:

* El adyacente al 1 es el 3 (debería ser el 2) => +2
* El adyacente al 2 es el 1 (debería ser el 3) => +2
* El adyacente al 4 es el 2 (debería ser el 1) => +2

In [310]:
sequential(Node((2,1,3,4,5,6,7,8,0)))

6

Por otro lado, para el estado inicial dado por:
    
              +---+---+---+
              | 1 | 2 | 3 |
              +---+---+---+
              | 4 | 5 | 0 |
              +---+---+---+
              | 7 | 8 | 6 |
              +---+---+---+
              
su puntuación es 3:

* El adyacente al 3 es el 0 (debería ser el 6) => +2
* El 6 ocupa la posición donde debería estar 0 => +1

In [311]:
sequential(Node((1,2,3,4,5,0,7,8,6)))

3

Finalmente, definimos la función de heurística:

In [307]:
def h3(node):
    return manhattan(node) + sequential(node)

Probamos la heurística para los estados iniciales anteriores: E1, E2, E3, E4.

In [308]:
resuelve_ocho_puzzle(E1, astar_search, h3)

Solución: ['Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: h3
Longitud de la solución: 17. Nodos analizados: 92


In [312]:
%%timeit
resuelve_ocho_puzzle(E1, astar_search, h3, False)

14 ms ± 1.43 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
## Búsqueda A* 'h3' E1
## 17 acciones
## 92 nodos analizados
## 14 milisegundos de media
## -------------------------------------------

In [313]:
resuelve_ocho_puzzle(E2, astar_search, h3)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: h3
Longitud de la solución: 11. Nodos analizados: 37


In [314]:
%%timeit
resuelve_ocho_puzzle(E2, astar_search, h3, False)

3.93 ms ± 64.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
## Búsqueda A* 'h3' E2
## 11 acciones
## 48 nodos analizados
## 3.93 milisegundos de media
## -------------------------------------------

In [315]:
resuelve_ocho_puzzle(E3, astar_search, h3)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: astar_search
Heurística: h3
Longitud de la solución: 20. Nodos analizados: 313


In [317]:
%%timeit
resuelve_ocho_puzzle(E3, astar_search, h3, False)

73.6 ms ± 1.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [299]:
## Búsqueda A* 'h3' E3
## 20 acciones
## 253 nodos analizados
## 76.1 milisegundos de media
## -------------------------------------------

In [318]:
resuelve_ocho_puzzle(E4, astar_search, h3)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: h3
Longitud de la solución: 3. Nodos analizados: 4


In [319]:
%%timeit
resuelve_ocho_puzzle(E4, astar_search, h3, False)

314 µs ± 15.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [300]:
## Búsqueda A* 'h3' E4
## 3 acciones
## 4 nodos analizados
## 314 microsegundos de media
## -------------------------------------------

#### Tabla de resultados actualizada.

|              Estado inicial |        | E1     | E2     | E3      | E4     |
|----------------------------:|:-------|-------:|-------:|--------:|-------:|
|                   _Anchura_ | **L**  | 17     | 11     | 20      | 3      |
|                             | **T**  | 15100  | 137    | 266000  | 0.162  |
|                             | **NA** | 13634  | 1255   | 57057   | 13     |
|               _Profundidad_ | **L**  | 7977   | NAN    | 39720   | 27     |
|                             | **T**  | 25500  | NAN    | 654000  | 0.868  |
|                             | **NA** | 8198   | NAN    | 42926   | 28     |
|            _Coste uniforme_ | **L**  | 17     | 11     | 20      | 3      |
|                             | **T**  | 136000 | 423    | 1471000 | 0.592  |
|                             | **NA** | 14092  | 870    | 48428   | 13     |
|    _Primero mejor (linear)_ | **L**  | 39     | 33     | 76      | 3      |
|                             | **T**  | 23.500 | 7.010  | 315     | 0.243  |
|                             | **NA** | 171    | 78     | 747     | 4      |
| _Primero mejor (manhattan)_ | **L**  | 49     | 29     | 36      | 3      |
|                             | **T**  | 174    | 7.170  | 118     | 0.187  |
|                             | **NA** | 522    | 75     | 401     | 118    |
|               _A* (linear)_ | **L**  | 17     | 11     | 20      | 3      |
|                             | **T**  | 404    | 7.670  | 5340    | 0.174  |
|                             | **NA** | 835    | 74     | 3230    | 4      |
|            _A* (manhattan)_ | **L**  | 17     | 11     | 20      | 3      |
|                             | **T**  | 43     | 1.620  | 289     | 0.219  |
|                             | **NA** | 245    | 24     | 732     | 4      |
|                   _A* (h3)_ | **L**  | 17     | 11     | 20      | 3      |
|                             | **T**  | 14     | 3.930  | 76.100  | 0.314  |
|                             | **NA** | 92     | 37     | 313     | 4      |

Sin embargo, esta heurística no es admisible. En efecto, volvamos al ejemplo anterior, con estado inicial:

              +---+---+---+
              | 1 | 2 | 3 |
              +---+---+---+
              | 4 | 5 | 0 |
              +---+---+---+
              | 7 | 8 | 6 |
              +---+---+---+
              
cuya puntuación con 'sequential' es 3. Por otro lado, todas las piezas están en su sitio menos 6, que está a una distancia de 1 de su posición, luego la puntuación con 'manhattan' es 1.

En definitiva, la puntuación con 'h3' es 4 (3 + 1), que es mayor que el coste real de llegar al estado objetivo: este coste es 1, pues basta con intercambiar el 6 con el hueco para resolver el puzzle.

Por tanto, la heurística sobreestima el coste real del estado; luego no es admisible.

#### Heurística más informada.

Dadas dos posiciones X,Y del puzzle y un camino de X a Y, se verifica:
* Si el camino tiene un número par de movimientos, entonces todo camino de X a Y tiene un número par de movimientos.
* Si el camino tiene un número impar de movimientos, entonces todo camino de X a Y tiene un número impar de movimientos.

Esto se debe a que los movimientos de una ficha sólo pueden ser de izquierda a derecha, y no en diagonal.

Por tanto, sean X e Y posiciones del camino tal que existe un único camino más corto que todos, entonces todo camino de X a Y distinto de ese tiene al menos 2 movimientos más. No obstante, en el tablero, el camino más corto no tiene por qué ser el único; en concreto, esto ocurre en determinados casos, en los que se dice que las piezas del tablero entran en conflicto.

Así, tomando el problema relajado en el que no incluimos todas las piezas en el tablero, construimos algunos ejemplos:

              +---+---+---+
              |   |   |   |
              +---+---+---+
       A >>   | 5 |   | 3 |
              +---+---+---+
              |   |   |   |
              +---+---+---+
              
Aquí, el 5 debe ocupar la posición del 3 y viceversa. Sin embargo, pese a que la distancia de ambos a su posición es 2, deben esquivarse para que cada uno ocupe su lugar; por tanto, el coste debe ser mayor que 4 (2 + 2). Uno de ellos hará un camino más largo, que añadirá 2 movimientos, por lo que el coste óptimo es 6.

              +---+---+---+
              |   |   |   |
              +---+---+---+
       B >>   | 4 | 3 |   |
              +---+---+---+
              |   |   |   |
              +---+---+---+
              
De forma parecida al anterior, sólo que ahora la suma de las distancias de Manhattan de ambos es 2; pero uno de ellos a de tomar un camino indirecto, aumentando el 2 el número de moviemientos, y dejando el coste en 4.

              +---+---+---+
              |   |   |   |
              +---+---+---+
       C >>   | 5 | 3 | 4 |
              +---+---+---+
              |   |   |   |
              +---+---+---+
     
Aquí, el 5 debe ir hacia arriba o hacia abajo para que el 3 y el 4 hagan movimientos equivalentes a su distancia de Manhattan (1), pero necesariamente el 5 hará 2 movimientos más (2 + 2), por lo que el coste es de 6 movimientos.

              +---+---+---+
              |   |   |   |
              +---+---+---+
       D >>   | 5 | 4 | 3 |
              +---+---+---+
              |   |   |   |
              +---+---+---+
              
Parecido al ejemplo A, aquí el 3 no puede ir directamente por su camino más corto, por lo que también ha de tomar un camino indirecto como el 5, lo que nos deja con un coste de 8 movimientos.

En base a estos conflictos, que generan situaciones en las que el coste se ve aumentado por encima de la heurística Manhattan, se puede llegar a una heurística admisible más informada, llamada de Conflicto Lineal.

Se dice que dos piezas t1 y t2 están en conflicto lineal si:
* Están en la misma línea (fila o columna).
* Las posiciones objetivo de ambas piezas están en esa misma línea.
* X está a la derecha de Y.
* La posición objetivo de X está a la izquierda de la posición objetivo de Y.

Con esta definición, podemos implementar la heurística que nos da el número mínimo de movimientos que se han de hacer para resolver los conflictos lineales de un estado; este número equivale a ir "eliminando" el elemento más conflictivo de cada línea hasta que no hay conflictos.

Como ya hemos visto, el caso peor es el ejemplo D, en el que están en conflicto:
* 5 y 4
* 5 y 3
* 4 y 3

En este caso, es necesario "eliminar" 2 elementos para acabar con los conflictos. El resto de casos con conflictos, se solucionan "eliminando" simplemente 1.

Por "eliminar", nos referimos a que una de las piezas ha de hacer necesariamente al menos 2 movimientos más, por lo que el valor de esta heurística será 2 por la suma del número de eliminados en cada fila y columna del tablero (más la heurística Manhattan).